In [222]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt 
import seaborn as sns

In [325]:
# Store paths 
cnvPath = 'CNVCalls.csv'
genotypePath = 'Genotyping Calls.csv'

#### Load and Clean CNV data ######

cnvDroppedCols = ['State', 'Confidence', 'CNV Length', 'Probe Density']

cnvCalls = pd.read_csv(cnvPath).drop(cnvDroppedCols, axis=1)

#### Load and Clean Genotype Call data ####

genotypeCalls = pd.read_csv(genotypePath, low_memory=False)
print(cnvCalls['Chromosome'].unique())

#genotypeCalls[genotypeCalls['Position'] == 172435007]

# Drop rows where there are no CNV calls
indexToDrop = (genotypeCalls[(genotypeCalls['Chromosome'] == 'X') | (genotypeCalls['Chromosome'] == 'Y') 
                              | (genotypeCalls['Chromosome'] == 'MT')]).index

genotypeCalls = genotypeCalls.drop(indexToDrop, axis=0)

# Convert Strings to integers in the Chromosome col
genotypeCalls['Chromosome'] = pd.to_numeric(genotypeCalls['Chromosome'])

# Check to make sure we did it correctly
print(genotypeCalls['Chromosome'].unique())

# Add Chromosome and position to be the first to columns of the genotypeCalls df
cols = genotypeCalls.columns.tolist()

cols = cols[-2:] + cols[:-2]

genotypeCalls = genotypeCalls[cols]

[ 1  2  3  4  5  6  8  9 10 11 12 13 14 16 17 18 19  7 15]
[ 1  6  4 15 10  7  9 18  2  5 17 19  8 12 13 11  3 14 16]


In [443]:
genotypeCalls[genotypeCalls['probeset_id'] == 'AX-170043934']


,Chromosome,Position,probeset_id,AxMouse_120919H_KHp1A01_1T,AxMouse_120919H_KHp1A02_2T,AxMouse_120919H_KHp1A03_3T,AxMouse_120919H_KHp1A04_4T,AxMouse_120919H_KHp1A05_5T,AxMouse_120919H_KHp1A06_6T,AxMouse_120919H_KHp1A07_7T,...,AxMouse_120919H_KHp1H03_2307E,AxMouse_120919H_KHp1H04_2305E,AxMouse_120919H_KHp1H05_2111E,AxMouse_120919H_KHp1H06_2109E,AxMouse_120919H_KHp1H07_2301E,AxMouse_120919H_KHp1H08_1301E,AxMouse_120919H_KHp1H09_2117E,AxMouse_120919H_KHp1H10_2121E,AxMouse_120919H_KHp1H11_2102E,AxMouse_120919H_KHp1H12_2106E
46474,1,188936236,AX-170043934,0,2,0,2,0,2,2,...,1,1,1,1,1,1,1,1,1,1


In [327]:
cnvCalls.head()

,Chromosome,Start,End,Sample,Start Probe,End Probe,Number of Markers
0,1,172435007,172538781,AxMouse_120919H_KHp1A01_1T,AX-170113402,AX-170303940,34
1,2,146184976,146565698,AxMouse_120919H_KHp1A01_1T,AX-170494042,AX-170299281,75
2,2,91046901,91114231,AxMouse_120919H_KHp1A01_1T,AX-170401949,AX-170174111,19
3,2,162131739,162218346,AxMouse_120919H_KHp1A01_1T,AX-170223417,AX-170168704,31
4,2,129540728,129625609,AxMouse_120919H_KHp1A01_1T,AX-170203438,AX-170128479,49


In [328]:
genotypeCalls.shape

(594475, 99)

In [335]:
genotypeCalls[genotypeCalls['probeset_id'] == 'AX-170299281']

,Chromosome,Position,probeset_id,AxMouse_120919H_KHp1A01_1T,AxMouse_120919H_KHp1A02_2T,AxMouse_120919H_KHp1A03_3T,AxMouse_120919H_KHp1A04_4T,AxMouse_120919H_KHp1A05_5T,AxMouse_120919H_KHp1A06_6T,AxMouse_120919H_KHp1A07_7T,...,AxMouse_120919H_KHp1H03_2307E,AxMouse_120919H_KHp1H04_2305E,AxMouse_120919H_KHp1H05_2111E,AxMouse_120919H_KHp1H06_2109E,AxMouse_120919H_KHp1H07_2301E,AxMouse_120919H_KHp1H08_1301E,AxMouse_120919H_KHp1H09_2117E,AxMouse_120919H_KHp1H10_2121E,AxMouse_120919H_KHp1H11_2102E,AxMouse_120919H_KHp1H12_2106E
301804,2,146565698,AX-170299281,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2


In [346]:
test = None
x = None

# for every CNV in cnvCalls 
count = 0

holdCalls = []
for _, row in cnvCalls.iterrows():
    
    # Get the Chromosome where the CNV is located, get the starting position of the CNV, and get the ending position
    # of the CNV
    chrom = row['Chromosome']
    start = row['Start']
    end = row['End']
    
    # Get all probes that are on the given chromosome
    data = genotypeCalls.loc[genotypeCalls['Chromosome'] == chrom]
    # Get all probes that span our CNV
    data = data.loc[(data['Position'] >= start) & (data['Position'] <= end)]
    
    test = data
    break
    # Get all of the sample names into a list
    samples = data.columns.tolist()[3:]
    # for each sample get the counts of each genotyping call for that probe
    count += 1
    # for each sample
    for sample in samples:
        # calculate the counts of each genotyping call
        x = data[sample].value_counts(False)
        # initialize list that hold potential genotyping calls
        potCalls = [-1, 0, 1, 2]
        # make a copy of this that we are going to modify
        copyPotCalls = [-1, 0, 1, 2]
        # for each potential call
        for call in potCalls:
            # if that call was not present in this sample set the value that corresponds
            # to the index of the call in copyPotCalls to 0 (no occurence)
            if call not in x.index:
                copyPotCalls[potCalls.index(call)] = 0
            # change the value at the index of copyPotCalls that corresponds to the current genotyping call
            # to the count of that call in the current sample
            else:
                copyPotCalls[copyPotCalls.index(call)] = x[call]
        
        # add the list of genotype call counts to a list
        
        toAdd = [x.name] + [chrom] + [start] + [end] + copyPotCalls
        
        holdCalls.append(toAdd)
        
        #print(holdCalls)
        
    # Now we have a series (x) that contains the counts of genotyping call and 
    # is indexed by the genotyping call itself as well as a DataFrame (test)
    # that contains all of the data for the CNV
    
    # Need to add the counts as a row to a new DataFrame but we need to see if any of 
    # the possible genotyping calls are missing and then set that to 0
    
    

In [341]:
z = pd.DataFrame(holdCalls, columns=['Sample', 'Chromosome', 'Start', 'End', -1, 0, 1, 2])

In [356]:
test

,Chromosome,Position,probeset_id,AxMouse_120919H_KHp1A01_1T,AxMouse_120919H_KHp1A02_2T,AxMouse_120919H_KHp1A03_3T,AxMouse_120919H_KHp1A04_4T,AxMouse_120919H_KHp1A05_5T,AxMouse_120919H_KHp1A06_6T,AxMouse_120919H_KHp1A07_7T,...,AxMouse_120919H_KHp1H03_2307E,AxMouse_120919H_KHp1H04_2305E,AxMouse_120919H_KHp1H05_2111E,AxMouse_120919H_KHp1H06_2109E,AxMouse_120919H_KHp1H07_2301E,AxMouse_120919H_KHp1H08_1301E,AxMouse_120919H_KHp1H09_2117E,AxMouse_120919H_KHp1H10_2121E,AxMouse_120919H_KHp1H11_2102E,AxMouse_120919H_KHp1H12_2106E
46137,1,172489388,AX-170043597,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46138,1,172490225,AX-170043598,-1,2,0,2,0,2,2,...,1,1,1,1,1,1,1,1,1,1
46139,1,172489036,AX-170043599,2,0,2,0,2,0,0,...,1,1,1,1,1,1,1,1,1,1
46140,1,172487433,AX-170043600,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
46141,1,172489471,AX-170043601,2,0,2,0,2,0,0,...,1,1,1,1,1,1,1,1,1,1
94325,1,172493185,AX-170091788,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
104481,1,172493080,AX-170101944,2,0,2,0,2,0,0,...,1,1,1,1,1,1,1,1,1,1
106237,1,172538538,AX-170103700,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128748,1,172482778,AX-170126213,0,2,0,2,0,2,2,...,1,1,1,1,1,1,1,1,1,1
184405,1,172495844,AX-170181872,0,2,0,2,0,2,2,...,1,1,1,1,1,1,1,1,1,1


In [358]:
test.iloc[:, :4]

,Chromosome,Position,probeset_id,AxMouse_120919H_KHp1A01_1T
46137,1,172489388,AX-170043597,0
46138,1,172490225,AX-170043598,-1
46139,1,172489036,AX-170043599,2
46140,1,172487433,AX-170043600,2
46141,1,172489471,AX-170043601,2
94325,1,172493185,AX-170091788,2
104481,1,172493080,AX-170101944,2
106237,1,172538538,AX-170103700,0
128748,1,172482778,AX-170126213,0
184405,1,172495844,AX-170181872,0


In [260]:
test

,Chromosome,Position,probeset_id,AxMouse_120919H_KHp1A01_1T,AxMouse_120919H_KHp1A02_2T,AxMouse_120919H_KHp1A03_3T,AxMouse_120919H_KHp1A04_4T,AxMouse_120919H_KHp1A05_5T,AxMouse_120919H_KHp1A06_6T,AxMouse_120919H_KHp1A07_7T,...,AxMouse_120919H_KHp1H03_2307E,AxMouse_120919H_KHp1H04_2305E,AxMouse_120919H_KHp1H05_2111E,AxMouse_120919H_KHp1H06_2109E,AxMouse_120919H_KHp1H07_2301E,AxMouse_120919H_KHp1H08_1301E,AxMouse_120919H_KHp1H09_2117E,AxMouse_120919H_KHp1H10_2121E,AxMouse_120919H_KHp1H11_2102E,AxMouse_120919H_KHp1H12_2106E
46137,1,172489388,AX-170043597,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
46138,1,172490225,AX-170043598,-1,2,0,2,0,2,2,...,1,1,1,1,1,1,1,1,1,1
46139,1,172489036,AX-170043599,2,0,2,0,2,0,0,...,1,1,1,1,1,1,1,1,1,1
46140,1,172487433,AX-170043600,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
46141,1,172489471,AX-170043601,2,0,2,0,2,0,0,...,1,1,1,1,1,1,1,1,1,1
94325,1,172493185,AX-170091788,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
104481,1,172493080,AX-170101944,2,0,2,0,2,0,0,...,1,1,1,1,1,1,1,1,1,1
106237,1,172538538,AX-170103700,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
128748,1,172482778,AX-170126213,0,2,0,2,0,2,2,...,1,1,1,1,1,1,1,1,1,1
184405,1,172495844,AX-170181872,0,2,0,2,0,2,2,...,1,1,1,1,1,1,1,1,1,1


In [261]:
x

1    19
2    10
0     4
Name: AxMouse_120919H_KHp1H12_2106E, dtype: int64

In [263]:
x.name

'AxMouse_120919H_KHp1H12_2106E'

In [429]:
test = None
x = None

# for every CNV in cnvCalls 
count = 0

holdCalls = []
for _, row in cnvCalls.iterrows():
    
    # Get the Chromosome where the CNV is located, get the starting position of the CNV, and get the ending position
    # of the CNV
    chrom = row['Chromosome']
    start = row['Start']
    end = row['End']
    sample = row['Sample']
    
    # Get all probes that are on the given chromosome
    data = genotypeCalls.loc[genotypeCalls['Chromosome'] == chrom]
    #test = data
    
    # Get all probes that span our CNV
    data = data.loc[(data['Position'] >= start) & (data['Position'] <= end)]
    
    
    # Get data for sample
    data = data[['Chromosome', 'Position', 'probeset_id', sample]]
    
    test = data
    
    # calculate the counts of each genotyping call
    x = data[sample].value_counts(False)
    # initialize list that hold potential genotyping calls
    potCalls = [-1, 0, 1, 2]
    # make a copy of this that we are going to modify
    copyPotCalls = [-1, 0, 1, 2]
    # for each potential call
    for call in potCalls:
        # if that call was not present in this sample set the value that corresponds
        # to the index of the call in copyPotCalls to 0 (no occurence)
        if call not in x.index:
            copyPotCalls[potCalls.index(call)] = 0
        # change the value at the index of copyPotCalls that corresponds to the current genotyping call
        # to the count of that call in the current sample
        else:
            copyPotCalls[copyPotCalls.index(call)] = x[call]
        # add the list of genotype call counts to a list
        #print(copyPotCalls)
    toAdd = [x.name] + [chrom] + [start] + [end] + copyPotCalls
        
    holdCalls.append(toAdd)
    
        
    #print(holdCalls)
        
    # Now we have a series (x) that contains the counts of genotyping call and 
    # is indexed by the genotyping call itself as well as a DataFrame (test)
    # that contains all of the data for the CNV
    
    # Need to add the counts as a row to a new DataFrame but we need to see if any of 
    # the possible genotyping calls are missing and then set that to 0
    
    

In [430]:
z = pd.DataFrame(holdCalls, columns=['Sample', 'Chromosome', 'Start', 'End', -1, 0, 1, 2])

In [435]:
z.to_csv('cleanData.csv', index=False)

In [432]:
z['Total'] = z[-1] + z[0] + z[1] + z[2]

In [433]:
z

,Sample,Chromosome,Start,End,-1,0,1,2,Total
0,AxMouse_120919H_KHp1A01_1T,1,172435007,172538781,1,13,0,19,33
1,AxMouse_120919H_KHp1A01_1T,2,146184976,146565698,27,0,1,47,75
2,AxMouse_120919H_KHp1A01_1T,2,91046901,91114231,6,0,0,13,19
3,AxMouse_120919H_KHp1A01_1T,2,162131739,162218346,1,17,0,13,31
4,AxMouse_120919H_KHp1A01_1T,2,129540728,129625609,24,0,1,24,49
...,...,...,...,...,...,...,...,...,...
4128,AxMouse_120919H_KHp1H12_2106E,12,113964955,114158110,20,10,1,11,42
4129,AxMouse_120919H_KHp1H12_2106E,13,43865609,43974077,9,0,0,14,23
4130,AxMouse_120919H_KHp1H12_2106E,17,27237763,27399528,6,0,11,23,40
4131,AxMouse_120919H_KHp1H12_2106E,18,60770184,60810651,6,0,0,2,8


In [439]:
sum(cnvCalls['Number of Markers'] != z['Total'])

2

In [440]:
np.where(cnvCalls['Number of Markers'] != z['Total'])

(array([   0, 2849]),)

In [441]:
z.loc[2849]

Sample        AxMouse_120919H_KHp1F10_7067P
Chromosome                                1
Start                             172138049
End                               172538781
-1                                       41
0                                         0
1                                         1
2                                        72
Total                                   114
Name: 2849, dtype: object

In [442]:
z.loc[0]

Sample        AxMouse_120919H_KHp1A01_1T
Chromosome                             1
Start                          172435007
End                            172538781
-1                                     1
0                                     13
1                                      0
2                                     19
Total                                 33
Name: 0, dtype: object